# Bllossom-8B를 이용한 한국어 LLM 튜토리얼
## 프롬프트 확정

## 01. 활용할 package 설정
 - GPU사용하기: colab에서 런타임 --> 런타임유형변경 --> T4 선택
 - 패키지설치: 아래 pip를 이용해 Transformers, accelerate 설치
 - 런타임재시작: 런타임 --> 세션다시시작  (accelerate설치 시 런타임 다시시작하셔야됩니다!)

In [1]:
!pip install transformers==4.40.0 accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 104.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.2.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.40.0 which is incompatible.


## 02. 모델준비

In [2]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-8B'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head)

## 03. 추론

### 결과 잘 나온 프롬프트 원본


PROMPT = '''You are a skilled storyteller and an advertising planner. Please answer the user's questions kindly. 당신은 유능한 이야기를 만들어내는 광고 기획자 입니다. 사용자의 질문에 대해 친절하게 답변해주세요.'''

instruction = '''
의자에 대해 스토리 형태로 자세 교정, 편안함을 강조해서 6개의 장면으로 나누어 광고해줘. 그런데 대본 형식 말고 상황, 인물, 카메라 구도를 나눠 장면을 설명하는 형식으로 해줘.
추상적인 단어들 말고 직관적이고 자세한 장면 설명이 필요해.

'''


In [ ]:
product = '커피'
request = '깊은 맛, 향기로운 커피향을 강조.'

PROMPT = '''You are a skilled storyteller and an advertising planner. Please answer the user's questions kindly. 당신은 유능한 이야기를 만들어내는 광고 기획자 입니다. 사용자의 질문에 대해 친절하게 답변해주세요.'''
instruction = f'''광고할 상품은 {product}야. 상품의 설명은 다음과 같아. {request}
                  6개의 장면으로 나누어 스토리 형태로 광고해줘. 각 장면은 150~160 token으로 제한할게.
                  그런데 대본 형식 말고 상황, 인물, 카메라 구도를 나눠 장면을 설명하는 형식으로 해줘.
                  추상적인 단어들 말고 직관적이고 자세한 장면 설명이 필요해.
                  '''

In [ ]:
messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{instruction}"}
    ]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=1024,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
    repetition_penalty = 1.1
)

results = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)
print(results)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


### 장면 1: 아침 일출
**장소:** 산림 근처의 언덕
**시간:** 새벽 5시
**인물:** 한 명의 여성 (30대 중반)
**카메라 구도:** 위에서 바라보는 전경샷
**설명:** 여자가 산책 중이며, 앞서 있는 언덕에서 일어나는 일출을 감상하고 있다. 하늘에는 붉은색과 주황색의 색채가 번지며, 이 모습은 자연 그대로의 아름다움을 담고 있다.

### 장면 2: 커피 포장 개봉
**장소:** 여자 집 안의 거실
**시간:** 아침 7시
**인물:** 여자가 다름 (여자)
**카메라 구도:** 내림차례 샷
**설명:** 여자는 아침 일정을 마치고, 거실 소파에 앉아 커피 포장을 열고 있다. 포장이 풀리면서 내부에 들어있는 커피콩들이 드러난다. 카메라는 포장지를 향해 내려오면서 포장의 매력과 재미를 강조한다.

### 장면 3: 커피 제조
**장소:** 여자 집 안의 부엌
**시간:** 아침 7시 15분
**인물:** 여자가 다름 (여자)
**카메라 구도:** cận접 촬영
**설명:** 여자는 부엌에서 커피머신에 커피콩을 넣고, 조리 시간을 설정하며 커피를 준비하고 있다. 카메라는 머신의 버튼을 누르는 순간을 포착하며, 커피의 제조 과정이 생생하게 보인다.

### 장면 4: 커피 향기
**장소:** 여자 집 안의 거실
**시간:** 아침 7시 20분
**인물:** 여자가 다름 (여자)
**카메라 구도:** 오버 더 숄더 샷
**설명:** 여자는 커피가 완성되자마자 거실로 돌아와, 커피냄새가 퍼지는 장면을 보여준다. 카메라는 그녀가 커피잔을 들고 마시는 순간까지 따라간다. 커피냄새가 풍기는 장면을 강조하여 관객들의 입안 가득 차오르게 만든다.

### 장면 5: 깊은 맛
**장소:** 여자 집 안의 거실
**시간:** 아침 7시 25분
**인물:** 여자가 다름 (여자)
**카메라 구도:** 내림차례 샷
**설명:** 여자는 커피를 마시며 눈을 감고 있으며, 깊은 만족감을 표현하고 있다. 카메라는 그녀의 얼굴을 바라보며, 커피의 깊은 맛을 느낀다는 그녀

## 4. 전처리

In [ ]:
import re

# '###' 이후의 텍스트만 남기기
if "###" in results:
  results = results.split("###", 1)[1]

# 원본 텍스트에서 `#`, `*`, 줄바꿈을 제거
cleaned_text = re.sub(r"[*]", "", results)
cleaned_text = re.sub(r"[\n]", ". ", cleaned_text)
cleaned_text = re.sub(r"\.\s*\.+", ".", cleaned_text)  # 점과 공백이 연속되는 패턴을 점 한 개로 변환

# 각 장면을 리스트로 저장
scenes = []
for line in cleaned_text.split("###"):
    line = line.strip()
    if line and line != ".":  # 빈 줄이 아닐 경우 + .밖에 있는 줄이 아닐 경우
        # 문장의 마지막 문자가 마침표가 아닌 경우
        if not line[-1].isalnum():
            line = line.rstrip(". ") + "."  # 기존 점을 삭제 후 마침표 추가
        elif not line.endswith("."):  # 마침표가 없으면 추가
            line += "."
        scenes.append(line)  # 리스트에 정리된 라인 추가

# 각 장면을 줄바꿈으로 구분하여 연결
text = "\n".join(scenes)
print(text)

장면 1: 아침 일출. 장소: 산림 근처의 언덕. 시간: 새벽 5시. 인물: 한 명의 여성 (30대 중반). 카메라 구도: 위에서 바라보는 전경샷. 설명: 여자가 산책 중이며, 앞서 있는 언덕에서 일어나는 일출을 감상하고 있다. 하늘에는 붉은색과 주황색의 색채가 번지며, 이 모습은 자연 그대로의 아름다움을 담고 있다.
장면 2: 커피 포장 개봉. 장소: 여자 집 안의 거실. 시간: 아침 7시. 인물: 여자가 다름 (여자). 카메라 구도: 내림차례 샷. 설명: 여자는 아침 일정을 마치고, 거실 소파에 앉아 커피 포장을 열고 있다. 포장이 풀리면서 내부에 들어있는 커피콩들이 드러난다. 카메라는 포장지를 향해 내려오면서 포장의 매력과 재미를 강조한다.
장면 3: 커피 제조. 장소: 여자 집 안의 부엌. 시간: 아침 7시 15분. 인물: 여자가 다름 (여자). 카메라 구도: cận접 촬영. 설명: 여자는 부엌에서 커피머신에 커피콩을 넣고, 조리 시간을 설정하며 커피를 준비하고 있다. 카메라는 머신의 버튼을 누르는 순간을 포착하며, 커피의 제조 과정이 생생하게 보인다.
장면 4: 커피 향기. 장소: 여자 집 안의 거실. 시간: 아침 7시 20분. 인물: 여자가 다름 (여자). 카메라 구도: 오버 더 숄더 샷. 설명: 여자는 커피가 완성되자마자 거실로 돌아와, 커피냄새가 퍼지는 장면을 보여준다. 카메라는 그녀가 커피잔을 들고 마시는 순간까지 따라간다. 커피냄새가 풍기는 장면을 강조하여 관객들의 입안 가득 차오르게 만든다.
장면 5: 깊은 맛. 장소: 여자 집 안의 거실. 시간: 아침 7시 25분. 인물: 여자가 다름 (여자). 카메라 구도: 내림차례 샷. 설명: 여자는 커피를 마시며 눈을 감고 있으며, 깊은 만족감을 표현하고 있다. 카메라는 그녀의 얼굴을 바라보며, 커피의 깊은 맛을 느낀다는 그녀의 표정과 동작을 강조한다. 커피 잔의 남은 액수가 줄어드는 모습도 함께 담는다.
장면 6: 하루의 시작. 장소: 여자 집 안의 거실. 시간: 아침 7시 30분. 인물: 여자가 

# 5. 함수로 만들기

In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import re

def llm(product, request):

  ## 모델 준비 ------------------------------------------
  model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-8B'

  tokenizer = AutoTokenizer.from_pretrained(model_id)
  model = AutoModelForCausalLM.from_pretrained(
      model_id,
      torch_dtype=torch.bfloat16,
      device_map="auto")
  model.eval()

  ## 모델 동작 ------------------------------------------

  PROMPT = '''You are a skilled storyteller and an advertising planner. Please answer the user's questions kindly. 당신은 유능한 이야기를 만들어내는 광고 기획자 입니다. 사용자의 질문에 대해 친절하게 답변해주세요.'''
  instruction = f'''광고할 상품은 {product}야. 상품의 설명은 다음과 같아. {request}
                    6개의 장면으로 나누어 스토리 형태로 광고해줘. 각 장면은 150~160 토큰으로 제한할게.
                    그런데 대본 형식 말고 상황, 인물, 카메라 구도를 나눠 장면을 설명하는 형식으로 해줘.
                    추상적인 단어들 말고 직관적이고 자세한 장면 설명이 필요해.
                    '''

  messages = [{"role": "system", "content": f"{PROMPT}"},
              {"role": "user", "content": f"{instruction}"}]

  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt").to(model.device)

  terminators = [tokenizer.eos_token_id,
                 tokenizer.convert_tokens_to_ids("<|eot_id|>")]

  outputs = model.generate(input_ids,
                           max_new_tokens=1024,
                           eos_token_id=terminators,
                           do_sample=True,
                           temperature=0.6,
                           top_p=0.9,
                           repetition_penalty = 1.1)

  results = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)

  ## 문자 전처리 ------------------------------------------
  # '###' 이후의 텍스트만 남기기
  if "###" in results:
    results = results.split("###", 1)[1]

  # 원본 텍스트에서 `#`, `*`, 줄바꿈을 제거
  cleaned_text = re.sub(r"[*]", "", results)
  cleaned_text = re.sub(r"[\n]", ". ", cleaned_text)
  cleaned_text = re.sub(r"\.\s*\.+", ".", cleaned_text)  # 점과 공백이 연속되는 패턴을 점 한 개로 변환

  # 각 장면을 리스트로 저장
  scenes = []
  for line in cleaned_text.split("###"):
    line = line.strip()
    if line and line != ".":  # 빈 줄이 아닐 경우 + .밖에 있는 줄이 아닐 경우
        # 문장의 마지막 문자가 마침표가 아닌 경우
        if not line[-1].isalnum():
            line = line.rstrip(". ") + "."  # 기존 점을 삭제 후 마침표 추가
        elif not line.endswith("."):  # 마침표가 없으면 추가
            line += "."
        scenes.append(line)  # 리스트에 정리된 라인 추가

  # 각 장면을 줄바꿈으로 구분하여 연결
  text = "\n".join(scenes)
  return text

# 6. 상품별로 결과 뽑기

In [3]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import re

def llm(product, request):

  # ## 모델 준비 ------------------------------------------
  # model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-8B'

  # tokenizer = AutoTokenizer.from_pretrained(model_id)
  # model = AutoModelForCausalLM.from_pretrained(
  #     model_id,
  #     torch_dtype=torch.bfloat16,
  #     device_map="auto")
  # model.eval()

  ## 모델 동작 ------------------------------------------

  PROMPT = '''You are a skilled storyteller and an advertising planner. Please answer the user's questions kindly. 당신은 유능한 이야기를 만들어내는 광고 기획자 입니다. 사용자의 질문에 대해 친절하게 답변해주세요.'''
  instruction = f'''광고할 상품은 {product}야. 상품의 설명은 다음과 같아. {request}
                    6개의 장면으로 나누어 스토리 형태로 광고해줘. 각 장면은 150~160 토큰으로 제한할게.
                    그런데 대본 형식 말고 상황, 인물, 카메라 구도를 나눠 장면을 설명하는 형식으로 해줘.
                    추상적인 단어들 말고 직관적이고 자세한 장면 설명이 필요해.
                    '''

  messages = [{"role": "system", "content": f"{PROMPT}"},
              {"role": "user", "content": f"{instruction}"}]

  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt").to(model.device)

  terminators = [tokenizer.eos_token_id,
                 tokenizer.convert_tokens_to_ids("<|eot_id|>")]

  outputs = model.generate(input_ids,
                           max_new_tokens=1024,
                           eos_token_id=terminators,
                           do_sample=True,
                           temperature=0.6,
                           top_p=0.9,
                           repetition_penalty = 1.1)

  results = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)

  ## 문자 전처리 ------------------------------------------
  # '###' 이후의 텍스트만 남기기
  if "###" in results:
    results = results.split("###", 1)[1]

  # 원본 텍스트에서 `#`, `*`, 줄바꿈을 제거
  cleaned_text = re.sub(r"[*]", "", results)
  cleaned_text = re.sub(r"[\n]", ". ", cleaned_text)
  cleaned_text = re.sub(r"\.\s*\.+", ".", cleaned_text)  # 점과 공백이 연속되는 패턴을 점 한 개로 변환

  # 각 장면을 리스트로 저장
  scenes = []
  for line in cleaned_text.split("###"):
    line = line.strip()
    if line and line != ".":  # 빈 줄이 아닐 경우 + .밖에 있는 줄이 아닐 경우
        # 문장의 마지막 문자가 마침표가 아닌 경우
        if not line[-1].isalnum():
            line = line.rstrip(". ") + "."  # 기존 점을 삭제 후 마침표 추가
        elif not line.endswith("."):  # 마침표가 없으면 추가
            line += "."
        scenes.append(line)  # 리스트에 정리된 라인 추가

  # 각 장면을 줄바꿈으로 구분하여 연결
  text = "\n".join(scenes)
  return text

In [4]:
coffee_product1 = '커피'
coffee_request1 = '깊은 맛, 향기로운 커피향을 강조.'

coffee_product2 = '커피'
coffee_request2 = '깊은 원두 향, 현대 직장인의 필수품이 커피란 것을 강조.'

mac_product1 = '맥북'
mac_request1 = '편한 기능, 예쁘고 현대적인 디자인, 빠른 속도를 강조.'

mac_product2 = '맥북'
mac_request2 = '큰 화면, 예쁜 디자인, 긴 배터리 사용 시간, 빠른 속도를 강조.'

iphone_product = '아이폰'
iphone_request = '많은 사람들이 사용, 카메라 기능, 예쁜 디자인을 강조'

conference_product1 = '컨퍼런스'
conference_request1 = '많은 사람들이 발표, 완벽한 결과물, 뜻깊은 경험을 강조'

conference_product2 = '컨퍼런스'
conference_request2 = '4개월 동안 준비한 프로젝트를 발표하는 날, 완벽한 결과물, 성취감을 강조'

In [5]:
llm(coffee_product1, coffee_request1)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'장면 1: 아침 일출. 위치: 산책로. 시간: 아침 일찍. 인물: 젊은 여자(30대 초반). 카메라 구도: 전경에서 촬영 (여자가 작은 배경으로 보임). 여자는 산책로를 걸으며 일몰과 맞물려 빛나는 하늘을 바라보고 있다. 그녀가 마주 보고 있는 산세로는 금빛이 가미된 아름다운 풍경이다. 이때부터 커피의 깊은 맛과 향기가 시작되는 것처럼, 여자의 미소가 점점 커진다.\n장면 2: 커피 가게. 위치: 작은 카페. 시간: 아침. 인물: 여자가 들어오는 가게 주인(40대 중반), 바리스타(20대 후반). 카메라 구도: 중앙에서 촬영 (여자가 가득 차 있는 카메라 프레임 안에 들어옴). 여자가 들어오자마자 주인이 웃으면서 "안녕하세요!"라고 인사한다. 바리스타가 빠르게 움직이며 커피 머신을 작동시키며 커피를 준비하고 있다. 여자가 주문하면, 바리스타는 조용히 "깊고 향기로운 커피가 드릴게요"라고 말하며 컵을 건네준다.\n장면 3: 커피 마시는 장면. 위치: 카페 테이블. 시간: 아침. 인물: 여자. 카메라 구도: 근접 촬영 (여자의 얼굴과 컵을 다 보여주는 화면). 여자가 컵을 들고 올린 순간, 카메라가 그녀의 입술이 컵을 만지작거리는 모습을 촬영. 그리고 그녀는 커피를 한 모금 마시면서 눈을 감고 만족스러운 표정을 지운다. 커피가 입속으로 사라지는 모습이 인상적으로 보인다.\n장면 4: 자연 속에서의 휴식. 위치: 공원. 시간: 오후. 인물: 남성(30대 중반)과 여자가 함께 앉아 있는 상태. 카메라 구도: 전체 촬영 (남녀가 함께 앉아 있는 테이블과 그 주변 환경). 남성과 여자가 서로 옆에 앉아 있는 모습. 여자가 남성에게 컵을 건넨 후, 남성은 커피를 마시며 행복한 표정을 짓는다. 둘 사이에는 따뜻한 분위기가 퍼져나가는 듯하다. 커피의 향기는 두 사람의 소통을 더 부드럽게 만든다.\n장면 5: 저녁 시간. 위치: 집의 거실. 시간: 저녁. 인물: 가족 구성원들 (부모와 자녀 2명). 카메라 구도: 중앙에서 촬영 (가족들이 모두 앉아 있는 테이블). 집안의 밝은

In [ ]:
## 커피
print('----- 1번')
llm(coffee_product1, coffee_request1)
print('----- 2번')
llm(coffee_product2, coffee_request2)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


----- 1번


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


----- 2번


"장면 1: 아침 출근길. 설명: 아침 일찍 출퇴근하는 직장인들이 차를 타거나 버스를 기다리는 장면. . 인물: 주인공 (30대 초반, 피부색: 밝은 회색), 다른 직장인들. . 카메라 구도: 위에서 내려다보는 bird's eye view. . 세부 사항: 주인공은 차 안에서 잠시 눈을 감고 있다가 시계를 확인하고, 컵을 들고 나오며, 다른 직장인들과 함께 버스나 택시에 타는 장면.\n장면 2: 사무실 내부. 설명: 회사 사무실 내부에서 직원들이 작업을 하고 있는 모습. . 인물: 주인공, 동료 직원들. . 카메라 구도: 중간 높이에서 촬영 (eye level shot). . 세부 사항: 주인공은 컴퓨터 앞에서 앉아 있는 상태로, 옆에는 커피잔과 노트가 놓여 있고, 동료들은 서로 대화하며 작업을 하고 있다.\n장면 3: 커피 마시는 순간. 설명: 주인공이 커피를 마시는 장면. . 인물: 주인공. . 카메라 구도: cận접 촬영 (close-up shot). . 세부 사항: 주인공이 커피 잔을 들어 올리고, 첫 번째 입술에 물어보는 순간을 집중적으로 찍음. 커피의 깊은 향이 코와 입에 퍼지는 듯한 묘사.\n장면 4: 업무 속에서의 에너지. 설명: 주인공이 업무를 할 때의 에너지가 증가하는 장면. . 인물: 주인공. . 카메라 구도: 전신 촬영 (full-body shot). . 세부 사항: 주인공이 커피를 마셨 후, 눈빛이 반짝이며 얼굴이 활기차게 변하며 업무를 더 집중되게 시작하는 모습.\n장면 5: 회의실 내에서의 성공. 설명: 회의실에서 발표를 하는 장면. . 인물: 주인공, 상사, 동료 직원들. . 카메라 구도: 상체 촬영 (upper body shot). . 세부 사항: 주인공이 백서를 들고 서서 발표하면서, 커피의 향이 그의 목소리에 어우러져 설득력 있는 말을 하게 되는 장면.\n장면 6: 저녁 출퇴근길. 설명: 저녁 시간에 출퇴근하는 직장인들이 차를 타거나 걸어가면서 커피를 마시는 장면. . 인물: 주인공, 다른 직장인들. . 카메라 구도: 위에서 

In [6]:
## 맥북
print('----- 1번')
llm(mac_product1, mac_request1)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


----- 1번


'장면 1: 아침 일과. 장소: 실내, 조용한 방. 인물: 젊은 여성(30대 초반). 카메라 구도: 전경에서 촬영 (여성의 얼굴과 상체를 보여줌). 여성이 잠에서 깨어나 침대에서 일어난다. 그녀는 잠옷 차림인데도 매우 아름답다. 침대 옆 책상을 열고, 그 안에는 맥북이 있다. . 사운드트랙: 부드러운 알람 소리가 끊긴 후, 조용한 방의 자연 소리 (침대가 울리는 소리 등).\n장면 2: 일과 준비. 장소: 같은 실내, 거실. 인물: 여성이 맥북을 들고 거실로 들어온다. 카메라 구도: 중간 샷 (여성 전체를 보여줌). 여성이 맥북을 들고 거실로 걸어온다. 거실은 깨끗하고 조용하며, 많은 책들이 놓여 있다. 그녀는 맥북을 열고, 몇 분 동안 집중한다. . 사운드트랙: 컴퓨터 키보드 클릭 소음, 가벼운 배경 음악.\n장면 3: 디자인 작업. 장소: 동일한 거실. 인물: 여성이 맥북 화면을 바라보고 작업 중. 카메라 구도: 가까운샷 (맥북 화면을 중심으로 여성의 얼굴과 손 movements를 보여줌). 여성이 디자인 프로그램을 켜고, 화면을 응시하며 작업을 진행한다. 그녀의 손가락이 마우스와 키보드를 빨리 타격하면서, 맥북 화면에서는 다양한 색깔과 패턴이 흐릿하게 움직인다. . 사운드트rack: 디자인 프로그램의 효과음, 가벼운 배경 음악.\n장면 4: 작업 완료. 장소: 여전히 동일한 거실. 인물: 여성이 작업을 끝낸 모습. 카메라 구도: 전경에서 촬영 (여성의 얼굴과 맥북을 보여줌). 여성이 작업을 끝냈고, 맥북을 닫는다. 그녀는 만족스러운 미소를 지으며, 맥북을 내려놓는다. 그녀의 눈빛은 행복하고 만족감을 표현한다. . 사운드트랙: 소프트한 "작업이 끝났습니다"라는 알림 소리.\n장면 5: 휴식 시간. 장소: 실내, 거실의 다른 부분. 인물: 여성이 맥북 없이 앉아 있는 모습. 카메라 구도: 전경에서 촬영 (여성이 앉아 있는 전체를 보여줌). 여성이 맥북 없이 거실의 한쪽에 앉아 책을 읽거나, 커피를 마신다. 그녀는 편안하고 즐거워 보인다. 거실은 여전히 깨끗하

In [7]:
print('----- 2번')
llm(mac_product2, mac_request2)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


----- 2번


'장면 1: 자연 속에서의 발견. 장소: 산책로에서 자연 풍경(숲, 호수 등). 인물: 젊은 여성, 가방에 맥북을 들고 걷는 중. 카메라 구도: 전경에서 여성과 맥북을 보여주는 wide shot. 설명: 여자가 천천히 산책하며 자연을 감상하고 있다. 그녀의 손에는 맥북이 들어 있으며, 그녀가 그걸 바라보고 있다.\n장면 2: 작업 공간의 변화. 장소: 집의 작은 작업실. 인물: 남성, 데스크탑 앞에 앉아 있는 상태에서 맥북을 열고 시작함. 카메라 구도: 전경에서 작업공간을 보여주는 wide shot -> 가까이 다가가서 맥북 화면을 보여주는 close-up shot. 설명: 남자친구가 작고 불편한 데스크탑을 접고 맥북을 열며 만족스러운 미소를 지으며 새로운 작업 환경을 즐긴다.\n장면 3: 여행에서의 자유. 장소: 비행기 내부 또는 공항. 인물: 부부, 각각 맥북을 열고 여행 중. 카메라 구도: 비행기 내부를 보여주는 wide shot -> 두 사람의 맥북 화면을 보여주는 split-screen. 설명: 부부가 서로 다른 나라로 여행 중에도 맥북을 통해 업무와 취미를 즐기며 자유롭게 생활하고 있다.\n장면 4: 배터리 수명을 보여주는 시퀀스. 장소: 다양한 환경(산, 집, 사무실 등)에서의 사용. 인물: 여러 가지 인물들이 맥북을 사용하는 모습. 카메라 구도: 각 인물마다 배터리 사용 시간을 보여주는 close-up shot. 설명: 다양한 환경에서 맥북을 사용하는 사람들이 배터리가 얼마나 오래 가는지 확인하는 장면. 배터리 잔량을 표시하는 화면을 반복적으로 보여준다.\n장면 5: 빠른 속도의 성과. 장소: 창업가의 사무실. 인물: 창업가, 맥북 앞에 앉아 작업 중. 카메라 구도: 창업가의 얼굴과 맥북 화면을 동시에 보여주는 two-shot. 설명: 창업가가 맥북을 통해 신속하게 데이터를 분석하고 결정을 내리는 과정을 보여준다. 화면이 빠르게 넘어가며 중요한 정보를 표시한다.\n장면 6: 휴식을 위한 엔딩. 장소: 야외의 편안한 자리. 인물: 여성이 맥북을 닫

In [8]:
## 의자
llm(iphone_product, iphone_request)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


"장면 1: 아침 일출. 장소: 바다 위의 작은 산책로. 시간: 새벽 5시. 인물: 한 남성이 sola를 들고 산책 중. 카메라 구도: 전경을 잡아, 사람의 뒷모습만 보여줌 (약 10초). - 설명: 화이트 밸런스를 통해 밝고 생동감 있는 색상을 표현. 바다와 하늘의 조화를 담아 자연의 아름다움을 감상케 함.\n장면 2: 카메라 기능. 장소: 같은 산책로. 시간: 새벽 5시 15분. 인물: 여전히 sola를 들고 있는 남성과 함께하는 친구들이 있음. 카메라 구도: 동그란 포커스로 카메라 모션 샷을 찍음 (약 20초). - 설명: 카메라의 초점을 맞추며, 카메라의 움직임을 통해 카메라의 고화질과 빠른 Autofocus 기능을 강조.\n장면 3: 예쁜 디자인. 장소: 도시 거리. 시간: 낮 12시. 인물: 다양한 연령대의 사람들이 다양한 아이폰 모델을 들고 다니는 모습. 카메라 구도: 평행선을 이루는 카메라 이동샷을 통해 여러 사람들을 촬영 (약 30초). - 설명: 다양한 모델과 색상의 아이폰을 다양하게 배치하여, 디자인의 다양성을 강조. 사람들의 다양한 생활 속에서 아이폰이 어떻게 활용되는지 보여줌.\n장면 4: 사용 빈도 높은 삶. 장소: 공원. 시간: 오후 3시. 인물: 가족과 친구들이 아이폰을 이용하며 즐거운 시간 보내기. 카메라 구도: 주변 환경과의 조화를 이루는 카메라 위치를 유지하며, 자연스러운 분위기 maintaine (약 40초). - 설명: 가족과 친구들이 아이폰을 통해 즐거운 순간을 나누는 장면을 통해, 아이폰이 일상생활의 중심이 된다는 점을 강조.\n장면 5: 카메라 기능 (세부 사항). 장소: 야외 스튜디오. 시간: 오후 4시. 인물: 사진작가와 모델. 카메라 구도: close-up 샷으로 카메라의 세부 기능을 강조 (약 25초). - 설명: 아이폰의 카메라 기능의 세밀함을 강조하기 위해, macro 촬영, 심도 효과 등을 보여주는 장면.\n장면 6: 결론 및 메시지. 장소: 다시 바다 위의 산책로. 시간: 저녁 8시. 인물: 처음 등장했

In [9]:
## 컨퍼런스
print('----- 1번')
llm(conference_product1, conference_request1)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


----- 1번


'장면 1: 준비 중인 회의실. 장소: 현대적인 컨벤션 센터 내의 큰 회의실  . 인물: 여러 명의 참가자들이 작은 테이블 주변에 앉아 노트와 가방을 풀고 있다. 카메라 구도: 위에서 내려다보는 조리개가 넓은 샷. . 여러 사람들의 소리가 들려오며, 회의실 안에는 다양한 색상의 노트북과 커피 잔들이 흩어져 있다. 참가자들은 서로 인사하며, 일부는 이미 노트를 펴고 자료를 검토하고 있다. 이 장면은 참가자들이 어떤 중요한 일을 준비 중이라는 느낌을 줍니다.\n장면 2: 발표자 준비. 장소: 회의실 뒤편의 준비 공간  . 인물: 한 명의 발표자가 자신의 발표 자료를 확인하고 있다. 카메라 구도: 가까운 샷으로 발표자를 찍음. . 발표자는 자신만의 개인 공간에 앉아, 노트북 화면을 바라보고 있다. 그는 자신의 발표 자료를 하나씩 확인하며, 때로는 미소를 지으며 고개를 끄덕입니다. 이 장면은 발표자의 준비와 자신감을 보여줍니다.\n장면 3: 첫 번째 발표 시작. 장소: 회의실 내의 무대  . 인물: 발표자와 청중  . 카메라 구도: 전신샷으로 발표자를 중심으로 청중을 포착함. . 발표자는 무대를 올라가서 마이크를 잡고, 청중의 시선을 맞춘다. 그는 자신만의 목소리로 이야기를 시작하며, 청중은 집중된 표정으로 그의 말을 듣고 있다. 회의실 안에는 손목시계의 클락 소리가 들리고, 청중의 눈빛이 깊어가는 모습이 보입니다.\n장면 4: 질의응답 시간. 장소: 회의실 내의 무대와 청중석 사이  . 인물: 발표자와 청중  . 카메라 구도: 다각샷으로 다양한 참가자들의 반응을 포착함. . 질의응답 시간이 시작되면서 청중석에서 손을 들어올리는 사람들이 있다. 발표자는 각 질문에 답하며, 청중은 그 대답을 경청하고 기록한다. 회의실 안에는 활기차고 열린 분위기가 퍼지고, 참가자들은 서로의 의견을 교환하는 모습이 보입니다.\n장면 5: 네트워킹 시간. 장소: 회의실 옆의 로비  . 인물: 참가자들이 모여 있는 그룹  . 카메라 구도: 가까운 샷으로 특정 그룹을 찍음. . 회의가 끝난 

In [10]:
print('----- 2번')
llm(conference_product2, conference_request2)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


----- 2번


'장면 1: 준비 중인 방.  장소: 집의 작업실.  시간: 오후 2시.  인물: 주인공(30대 남성).  카메라 구도: overhead shot.  설명: 방은 책상과 의자, 컴퓨터와 노트북으로 가득 차 있는 workspace이다. 주인공은 손에는 마커를 쥐고, 벽에 붙여진 타임라인을 점검하고 있다. 방은 조명이 밝고, 에어컨이 시원한 느낌을 준다.\n장면 2: 프로젝트 진행 중.  장소: 같은 작업실.  시간: 밤 11시.  인물: 주인공(30대 남성), 동료(20대 여성).  카메라 구도: medium shot.  설명: 주인공은 컴퓨터 앞에서 코드를 작성하고 있다. 동료는 옆에서 자료를 검토하며 지침을 주고 있다. 두 사람은 서로의 얼굴 근육을 살펴주고 있으며, 집중된 표정이다. 배경에는 종이 접힌 파일과 파란색 커튼이 보인다.\n장면 3: 마지막 준비.  장소: 회의실.  시간: 아침 9시.  인물: 주인공(30대 남성), 다른 참가자들.  카메라 구도: wide shot.  설명: 회의실은 큰 화이트보드와 복잡한 다이어그램, 그리고 여러 사람들이 앉아 있는 모습이다. 주인공은 백패지를 준비하고 있다. 모든 사람들은 긴장한 표정을 짓고 있다. 회의실은 자연광이 들어오는 창문 때문에 밝고 활기찬 느낌을 준다.\n장면 4: 발표 시작.  장소: 대회의실.  시간: 오후 2시.  인물: 주인공(30대 남성), 대규모 청중.  카메라 구도: close-up.  설명: 주인공은 무대 위에서 마이크를 잡고 있다. 그의 눈빛은 자신감 있게 빛나며, 입술은 미세하게 움직이고 있다. 청중석에는 다양한 옷차림의 사람들이 앉아 있다. 뒷배경은 큰 스크린과 projector가 설치되어 있다.\n장면 5: 발표 완료.  장소: 여전히 대회의실.  시간: 오후 2시.  인물: 주인공(30대 남성), 청중.  카메라 구도: wide shot.  설명: 주인공은 발표를 마친 후 무대를 내려오고 있다. 청중석에서는 박수와 환호성이 터져 나오고 있다. 주인공의 얼굴에는 웃음과 자부심